In [1]:


import warnings
warnings.simplefilter(action ="ignore")

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Import the necessary packages
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Algorithms
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve

In [2]:
# Load Dataset
data = pd.read_csv('C:/Users/Dhruv/Downloads/Maternal Health Risk Data Set/fetal_health.csv')

In [3]:
columns = ['baseline value', 'accelerations', 'fetal_movement',
       'uterine_contractions', 'light_decelerations', 'severe_decelerations',
       'prolongued_decelerations', 'abnormal_short_term_variability',
       'mean_value_of_short_term_variability',
       'percentage_of_time_with_abnormal_long_term_variability',
       'mean_value_of_long_term_variability', 'histogram_width',
       'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
       'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
       'histogram_median', 'histogram_variance', 'histogram_tendency']
scale_X = StandardScaler()
X =  pd.DataFrame(scale_X.fit_transform(data.drop(["fetal_health"],axis = 1),), columns = columns)

In [24]:
import pickle
scaler_filename = "scaler.pkl"
pickle.dump(scale_X, open(scaler_filename, 'wb'))

In [4]:
X.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency
0,-1.352220,-0.822388,-0.20321,-1.482465,-0.638438,-0.057476,-0.268754,1.513190,-0.943095,1.802542,...,-0.165507,-1.068562,-2.119592,-0.701397,-0.458444,-1.065614,0.153270,-1.181642,1.870569,1.112980
1,-0.132526,0.730133,-0.20321,0.554627,0.375243,-0.057476,-0.268754,-1.744751,0.868841,-0.535361,...,1.529124,-0.865539,1.893794,0.655137,0.958201,0.216638,0.089126,0.132038,-0.234998,-0.524526
2,-0.030884,-0.046128,-0.20321,1.233657,0.375243,-0.057476,-0.268754,-1.802928,0.868841,-0.535361,...,1.529124,-0.865539,1.893794,0.316003,0.958201,0.216638,0.024982,-0.006244,-0.200481,-0.524526
3,0.070757,-0.046128,-0.20321,1.233657,0.375243,-0.057476,-0.268754,-1.802928,1.208579,-0.535361,...,1.195333,-1.373097,0.333033,2.350804,-0.458444,-0.027600,-0.039162,-0.075385,-0.200481,1.112980
4,-0.132526,0.988886,-0.20321,1.233657,-0.638438,-0.057476,-0.268754,-1.802928,1.208579,-0.535361,...,1.195333,-1.373097,0.333033,1.672537,-0.458444,-0.027600,0.089126,-0.006244,-0.269516,1.112980


In [5]:
y = data["fetal_health"]

In [6]:
# Importing train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1488, 21), (638, 21), (1488,), (638,))

In [8]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score


Gradient Boosting classifier (GBC)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pipeline_rf = Pipeline([("scaler", StandardScaler()),
                        ("rf_classifier", RandomForestClassifier())])

# check list of available parameters using [estimator].get_params().keys()
# e.g. "print(pipeline_rf.get_params().keys())"

params = {
    'rf_classifier__n_estimators': [200],
    #'rf_classifier__max_features': ['auto', 'sqrt', 'log2'],
    'rf_classifier__max_depth' : [16],
    'rf_classifier__criterion' :['entropy'],
    'rf_classifier__n_jobs':[-1]
}

clf = GridSearchCV(pipeline_rf, param_grid=params, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_)

# print(opt_params)



{'rf_classifier__criterion': 'entropy', 'rf_classifier__max_depth': 16, 'rf_classifier__n_estimators': 200, 'rf_classifier__n_jobs': -1}
0.9415271281043094


In [18]:
# cleaning best_params_
opt_params = {}
for item in clf.best_params_.items():
    opt_params[item[0][15:]]=item[1]
# print(opt_params)

# construct optimized random forest classifier
rf_opt = RandomForestClassifier(**opt_params)
rf_opt.fit(X_train, y_train)
y_pred = rf_opt.predict(X_test)



In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="micro")

print("Accuracy: {}". format(accuracy))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("f1: {}".format(f1))

Accuracy: 0.9247648902821317
Precision: 0.9216465443890642
Recall: 0.9247648902821317
f1: 0.9247648902821317


In [20]:
import pickle

In [23]:
scaler_filename = "scaler.pkl"
pickle.dump(scale_X, open(scaler_filename, 'wb'))